In [2]:
import os
os.getcwd()
# os.chdir('..')
print(f'Current working directory is {os.getcwd()}')

Current working directory is c:\Users\blond\Documents\NssProjects\sleigh_the_model


In [26]:
import pandas as pd
import scripts.features as ft
import scripts.models as md
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor

In [4]:
file = 'data/dataformodel.csv'

In [5]:
months_in_adv = 3

In [6]:
data = ft.load_data(file, months_in_adv)

In [7]:
features_to_lag = [
    'pce',
    'inflation',
    'unemployment_rate',
    'revolving_credit',
    'nonrevolving_credit',
    'total_credit'
]

In [8]:
data_with_lag = ft.add_lag_features(data, months_in_adv, features_to_lag)

In [9]:
data_with_lag = data_with_lag.dropna()

In [10]:
X = data_with_lag.drop(columns=['date', 'pce_future'])
y = data_with_lag['pce_future']

In [11]:
test_size = 60
min_train_size = 60
total_months = len(data_with_lag)
max_splits = (total_months - min_train_size) // test_size
print(max_splits)
n_splits = max_splits

12


In [12]:
time_series_split = TimeSeriesSplit(
    n_splits = n_splits,
    test_size = test_size
)

In [13]:
lg = LinearRegression() 

In [15]:
lr_mse_scores, lr_r2_scores, lr_test_results = md.train_eval_model(X, y, lg, time_series_split, data_with_lag, 'Linear Regression')

*** Linear Regression ***

--- Split 1 ---
  TRAIN: 70 months (1959-04 to 1965-01)
  TEST: 60 months (1965-02 to 1970-01)
Model Performance:
    Mean Squared Error:  95.99
    R-squared:  0.97
    Actual PCE: 636.00
    Predicted PCE: 628.65

--- Split 2 ---
  TRAIN: 130 months (1959-04 to 1970-01)
  TEST: 60 months (1970-02 to 1975-01)
Model Performance:
    Mean Squared Error:  110.05
    R-squared:  0.99
    Actual PCE: 995.00
    Predicted PCE: 990.68

--- Split 3 ---
  TRAIN: 190 months (1959-04 to 1975-01)
  TEST: 60 months (1975-02 to 1980-01)
Model Performance:
    Mean Squared Error:  273.99
    R-squared:  0.99
    Actual PCE: 1695.20
    Predicted PCE: 1713.65

--- Split 4 ---
  TRAIN: 250 months (1959-04 to 1980-01)
  TEST: 60 months (1980-02 to 1985-01)
Model Performance:
    Mean Squared Error:  726.80
    R-squared:  0.99
    Actual PCE: 2659.50
    Predicted PCE: 2699.76

--- Split 5 ---
  TRAIN: 310 months (1959-04 to 1985-01)
  TEST: 60 months (1985-02 to 1990-01)
Mod

From looking at the metrics the Linear Regression model preformed the best of the ones tested

In [16]:
xgbr = xgb.XGBRegressor(
    objective = 'reg:squarederror', 
    n_estimators = 100, 
    learning_rate = 0.1, 
    max_depth = 6, 
    random_state = 42
) 

In [ ]:
xgbr_mse_scores, xgbr_r2_scores, xgbr_test_results = md.train_eval_model(X, y, xgbr, time_series_split, data_with_lag, 'XGB Regressor')

*** XGBRegressor ***

--- Split 1 ---
  TRAIN: 70 months (1959-04 to 1965-01)
  TEST: 60 months (1965-02 to 1970-01)
Model Performance:
    Mean Squared Error:  12504.12
    R-squared: -2.52
    Actual PCE: 636.00
    Predicted PCE: 436.79

--- Split 2 ---
  TRAIN: 130 months (1959-04 to 1970-01)
  TEST: 60 months (1970-02 to 1975-01)
Model Performance:
    Mean Squared Error:  39588.36
    R-squared: -2.39
    Actual PCE: 995.00
    Predicted PCE: 635.49

--- Split 3 ---
  TRAIN: 190 months (1959-04 to 1975-01)
  TEST: 60 months (1975-02 to 1980-01)
Model Performance:
    Mean Squared Error:  162878.38
    R-squared: -2.79
    Actual PCE: 1695.20
    Predicted PCE: 993.27

--- Split 4 ---
  TRAIN: 250 months (1959-04 to 1980-01)
  TEST: 60 months (1980-02 to 1985-01)
Model Performance:
    Mean Squared Error:  295277.67
    R-squared: -2.95
    Actual PCE: 2659.50
    Predicted PCE: 1699.12

--- Split 5 ---
  TRAIN: 310 months (1959-04 to 1985-01)
  TEST: 60 months (1985-02 to 1990-01

Looking at metrics XGB Regressor model did not out preform Linear Regression  

In [21]:
rfr = RandomForestRegressor(
    random_state=42
)

In [22]:
rfr_mse_scores, rfr_r2_scores, rfr_test_results = md.train_eval_model(X, y, rfr, time_series_split, data_with_lag, 'Random Forest Regressor')

*** Random Forest Regressor ***

--- Split 1 ---
  TRAIN: 70 months (1959-04 to 1965-01)
  TEST: 60 months (1965-02 to 1970-01)
Model Performance:
    Mean Squared Error:  12958.01
    R-squared: -2.65
    Actual PCE: 636.00
    Predicted PCE: 434.42

--- Split 2 ---
  TRAIN: 130 months (1959-04 to 1970-01)
  TEST: 60 months (1970-02 to 1975-01)
Model Performance:
    Mean Squared Error:  62016.50
    R-squared: -4.31
    Actual PCE: 995.00
    Predicted PCE: 576.12

--- Split 3 ---
  TRAIN: 190 months (1959-04 to 1975-01)
  TEST: 60 months (1975-02 to 1980-01)
Model Performance:
    Mean Squared Error:  166250.96
    R-squared: -2.87
    Actual PCE: 1695.20
    Predicted PCE: 988.85

--- Split 4 ---
  TRAIN: 250 months (1959-04 to 1980-01)
  TEST: 60 months (1980-02 to 1985-01)
Model Performance:
    Mean Squared Error:  303350.17
    R-squared: -3.06
    Actual PCE: 2659.50
    Predicted PCE: 1690.58

--- Split 5 ---
  TRAIN: 310 months (1959-04 to 1985-01)
  TEST: 60 months (1985-02

Looking at metrics Random Forest Regressor model did not out preform Linear Regression  

In [27]:
cat = CatBoostRegressor(
    iterations=100,
    learning_rate=0.1,
    depth=6,
    random_state=42,
    verbose=0
)

In [28]:
cat_mse_scores, cat_r2_scores, cat_test_results = md.train_eval_model(X, y, cat, time_series_split, data_with_lag, 'Cat Boost Regressor')

*** Cat Boost Regressor ***

--- Split 1 ---
  TRAIN: 70 months (1959-04 to 1965-01)
  TEST: 60 months (1965-02 to 1970-01)
Model Performance:
    Mean Squared Error:  13543.71
    R-squared: -2.82
    Actual PCE: 636.00
    Predicted PCE: 431.44

--- Split 2 ---
  TRAIN: 130 months (1959-04 to 1970-01)
  TEST: 60 months (1970-02 to 1975-01)
Model Performance:
    Mean Squared Error:  75855.77
    R-squared: -5.50
    Actual PCE: 995.00
    Predicted PCE: 531.81

--- Split 3 ---
  TRAIN: 190 months (1959-04 to 1975-01)
  TEST: 60 months (1975-02 to 1980-01)
Model Performance:
    Mean Squared Error:  174000.41
    R-squared: -3.05
    Actual PCE: 1695.20
    Predicted PCE: 989.17

--- Split 4 ---
  TRAIN: 250 months (1959-04 to 1980-01)
  TEST: 60 months (1980-02 to 1985-01)
Model Performance:
    Mean Squared Error:  460901.99
    R-squared: -5.17
    Actual PCE: 2659.50
    Predicted PCE: 1593.37

--- Split 5 ---
  TRAIN: 310 months (1959-04 to 1985-01)
  TEST: 60 months (1985-02 to 

Looking at metrics Cat Boost Regressor model did not out preform Linear Regression 

In [33]:
lgbm = lgb.LGBMRegressor(
    objective='regression',
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)

Tried lgbm, there was multiple warnings of no further splits with positive gain 